In [3]:
import pandas as pd
import numpy as np
from math import sqrt
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from pymlx import *

from keras.models import Sequential, Model
from keras.layers import Activation, Input, Dense, Conv1D, MaxPooling1D, AveragePooling1D, Dropout, Flatten, GRU, LSTM
from keras.layers.advanced_activations import ELU
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import *
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

K.set_image_dim_ordering('th')

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def compute_mae(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],dense)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    result = mean_absolute_error(y_pred=forecasts, y_true=y_denom)
    return result


def compute_mape_centerScale(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],1)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    sub_result = np.abs((y_denom - forecasts) / y_denom )
    result = (np.mean(sub_result))
    return result


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))


def test_mae(model):
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)
    return(compute_mae(model, X_train, y_train), compute_mae(model, X_test, y_test))


def report(train, test):
    train_mean = np.asarray(train).mean()
    train_std = np.asarray(train).std()
    test_mean = np.asarray(test).mean()
    test_std = np.asarray(test).std()
    train = str('Training MAE %f ± %f'% (train_mean, train_std))
    test = str('Testing MAE %f ± %f'% (test_mean, test_std))
    return(train, test)

In [4]:
# ndre_data = pd.read_csv('../HydroSatML/data/final_join_subbed_missing_soilM.csv')
# ndre_data_sub_30 = pd.read_csv('../HydroSatML/data/final_join_subbed_bare_soil.csv')
# ndre_data_sub_40 = pd.read_csv('../HydroSatML/data/final_join_subbed_bare_soil_40.csv')
ndre_data_lag_40 = pd.read_csv('../data/final_join_subbed_bare_soil_with_lag_40_2.csv')


In [5]:
ndre_data_lag_40.columns

Index(['field', 'average_adjacent', 'stdev_adjacent', 'sensor', 'date',
       'sensor_full_name', 'depth_1', 'depth_2', 'depth_3', 'year',
       'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s',
       'irradiance.w_per_m.2', 'sand_1', 'sand_2', 'sand_3', 'silt_1',
       'silt_2', 'silt_3', 'clay_1', 'clay_2', 'clay_3', 'l3_depth_1',
       'l3_depth_2', 'l3_depth_3', 'l5_depth_1', 'l5_depth_2', 'l5_depth_3',
       'l7_depth_1', 'l7_depth_2', 'l7_depth_3', 'avg_soilM', 'avg_soilM_l3',
       'avg_soilM_l5', 'avg_soilM_l7', 'avg_clay', 'avg_sand', 'avg_silt'],
      dtype='object')

In [6]:
# keep_cols = ['avg_soilM', 'average_adjacent', 'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 
#              'irradiance.w_per_m.2', 'sand_1', 'sand_2', 'sand_3', 'silt_1', 'silt_2', 'silt_3', 'clay_1', 
#              'clay_2', 'clay_3']
keep_cols = ['avg_soilM_l3', 'average_adjacent', 'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 
             'irradiance.w_per_m.2', 'avg_clay', 'avg_sand', 'avg_silt']

dataset = ndre_data_lag_40[keep_cols]

In [7]:
X = dataset.iloc[:,1:]
X = X.values

y = dataset.iloc[:,0]
y = y.values

# split data into train and test sets
seed = 7
test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

#### normalize

mean_X = X_train.mean(axis=0)
std_X = X_train.std(axis=0)

X_train = (X_train - mean_X) / std_X
X_test = (X_test - mean_X) / std_X

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

mean_y = y_train.mean(axis=0)
std_y = y_train.std(axis=0)

y_train = (y_train - mean_y) / std_y
y_test = (y_test - mean_y) / std_y

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

dim = X_train.shape[1]

### Building the NN

In [10]:
np.random.seed(84)
tf.set_random_seed(84)

def create_NN_model(units1=16,
                    units2=16,
                    units3=16,
                    units4=16,
                    layers=1,
                    epochs=50,
                    optimizer='adam',
                    activation=ELU(),
                    beta_1=0.9,
                    beta_2=0.99,
                    epsilon=1e-8,
                    decay=0,
                    schedule_decay=0.004,
                    rho=0.8,
                    lr=0.001,
                    momentum=0.9,
                    nesterov=False):
    
    if layers == 1:
        model = Sequential()
        model.add(Dense(units1, input_dim=dim, activation=activation))
        model.add(Dense(1, activation=activation))
    elif layers == 2:
        model = Sequential()
        model.add(Dense(units1, input_dim=dim, activation=activation))
        model.add(Dense(units2, activation=activation))
        model.add(Dense(1, activation=activation))
    elif layers == 3:
        model = Sequential()
        model.add(Dense(units1, input_dim=dim, activation=activation))
        model.add(Dense(units2, activation=activation))
        model.add(Dense(units3, activation=activation))
        model.add(Dense(1, activation=activation))
    elif layers == 4:
        model = Sequential()
        model.add(Dense(units1, input_dim=dim, activation=activation))
        model.add(Dense(units2, activation=activation))
        model.add(Dense(units3, activation=activation))
        model.add(Dense(units4, activation=activation))
        model.add(Dense(1, activation=activation))
    else:
        pass
    
    if(optimizer == 'SGD'):
        sgd = SGD(lr=lr, momentum=momentum, decay=decay, nesterov=nesterov)
    elif(optimizer == 'RMSprop'):
        sgd = RMSprop(lr=lr, rho=rho)
    elif(optimizer == 'Nadam'):
        sgd = Nadam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, schedule_decay=schedule_decay)
    elif(optimizer == 'Adagrad'):
        sgd = Adagrad(lr=lr, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adadelta'):
        sgd = Adadelta(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adam'):
        sgd = Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adamax'):
        sgd = Adamax(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    else:
        pass
    
    model.compile(loss='mae', optimizer=sgd, metrics=['mse'])
    
    return model

### Setting paramters

In [16]:
model = KerasRegressor(build_fn=create_NN_model, shuffle=True, verbose=0)

units1 = [4,8,26,32,64]
units2 = [4,8,16,32,64]
units3 = [4,8,16,32,64]
units4 = [4,8,16,32,64]

layers = [1,2,3,4]
epochs = [100]
batch_size = [1,2,3,4]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['relu', ELU()]
decay = np.arange(0,.1,0.001).tolist()
beta_1 = np.random.uniform(low=0.90, high=1.0, size=20).tolist()
beta_2 = np.random.uniform(0.90, 1.0, 20).tolist()
rho = np.random.uniform(0.80, 0.95, 20).tolist()
epsilon = [1e-09, 1e-08,1e-07, 1e-06, 1e-05, 1e-04]
schedule_decay = np.arange(0.001,0.01,0.0005).tolist()
lr = [10**k for k in range(-5, -1)]

param_grid = dict(units1=units1,
                 units2=units2,
                 units3=units3,
                 units4=units4,
                 layers=layers,
                 epochs=epochs,
                 optimizer=optimizer,
                 activation=activation,
                 lr=lr,
                 schedule_decay=schedule_decay,
                 epsilon=epsilon,
                 rho=rho,
                 beta_1=beta_1,
                 beta_2=beta_2,
                 decay=decay)

In [17]:
start_timing()
sweeper = random_sweep(
    X_train, y_train, 
    model, param_grid,
    scoring=compute_mae, 
    n_iter=75, n_jobs=1, 
    refit=False, cv=3, verbose=1)
report_timing()

Fitting 3 folds for each of 75 candidates, totalling 225 fits
Elapsed: 31 minutes


[Parallel(n_jobs=1)]: Done 225 out of 225 | elapsed: 31.5min finished


In [13]:
results = sweep_stats(sweeper)
# FILE_PATH = os.path.join(CWD, "../results/best_params/forecasting/tCNN_hourlyData-25percent_SEEDFEATS-DP-RH-WindGust-Speed_Resp-ForeCast-LW-8-2-VWC_H1_Rw120_W360_Tune_run1.csv")
# results.to_csv(FILE_PATH)
results

,Score,Std,activation,beta_1,beta_2,decay,epochs,epsilon,layers,lr,optimizer,rho,schedule_decay,units1,units2,units3,units4
0,0.048359,0.001738,<keras.layers.advanced_activations.ELU object ...,0.965180,0.926673,0.096,100,1.000000e-09,1,0.00010,Adadelta,0.842714,0.0060,32,16,8,16
1,0.046185,0.003566,relu,0.953561,0.949834,0.087,100,1.000000e-04,1,0.00001,Adam,0.867730,0.0035,8,32,32,8
2,0.043349,0.005280,relu,0.910583,0.983947,0.053,100,1.000000e-08,2,0.00010,Adagrad,0.861757,0.0085,4,4,64,16
3,0.042719,0.003553,<keras.layers.advanced_activations.ELU object ...,0.913059,0.916112,0.074,100,1.000000e-05,4,0.00010,Adamax,0.884929,0.0015,32,32,16,4
4,0.042378,0.001510,relu,0.925831,0.913841,0.037,100,1.000000e-09,2,0.00001,Adamax,0.856582,0.0075,64,64,4,64
5,0.041232,0.001212,relu,0.929739,0.916112,0.033,100,1.000000e-09,2,0.00010,Adamax,0.904762,0.0030,26,32,4,8
6,0.040706,0.001426,<keras.layers.advanced_activations.ELU object ...,0.962848,0.933100,0.081,100,1.000000e-09,3,0.00010,Adamax,0.818046,0.0015,8,4,8,8
7,0.040673,0.001368,relu,0.937024,0.935627,0.047,100,1.000000e-09,4,0.00010,SGD,0.823678,0.0070,32,4,64,32
8,0.040195,0.000918,relu,0.953561,0.963807,0.013,100,1.000000e-05,2,0.00100,RMSprop,0.821949,0.0035,4,8,16,4
9,0.039959,0.002243,relu,0.904604,0.907418,0.065,100,1.000000e-06,4,0.00010,RMSprop,0.903009,0.0085,32,16,32,32


In [14]:
for i in range(15):

    RUN = i+1
    print('Best Run Num: {0}'.format(RUN))
    params = results.iloc[-RUN,2:].to_dict()
    # params

    ### Testing the best params

    np.random.seed(84)
    tf.set_random_seed(84)

    params['epochs'] = 100

    train_results, test_results = [], []
    ITERATIONS=3
    VERBOSE=0

    # params

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-08, mode='auto', verbose=1)

    for i in range(ITERATIONS):
        model = KerasRegressor(build_fn=create_NN_model, shuffle=True, verbose=VERBOSE, **params)
    #     history = model.fit(train_X, train_y, verbose=VERBOSE, validation_split = 0.15, callbacks=[early_stopping])
        history = model.fit(X_train, y_train, verbose=VERBOSE, validation_split = None)
        train_results.append(test_mae(model)[0])
        test_results.append(test_mae(model)[1])
    report(train_results, test_results)

Best Run Num: 1


('Training MAE 0.032017 ± 0.000126', 'Testing MAE 0.037196 ± 0.000458')

Best Run Num: 2


('Training MAE 0.039627 ± 0.000667', 'Testing MAE 0.040826 ± 0.000178')

Best Run Num: 3


('Training MAE 0.039082 ± 0.001374', 'Testing MAE 0.040706 ± 0.000266')

Best Run Num: 4


KeyboardInterrupt: 

In [ ]:
print('test')

In [ ]:
RUN = 1

params = results.iloc[-RUN,2:].to_dict()

### Testing the best params

np.random.seed(84)
tf.set_random_seed(84)

params['epochs'] = 100

train_results, test_results = [], []
ITERATIONS=3
VERBOSE=0

# params

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-08, mode='auto', verbose=1)

for i in range(ITERATIONS):
    model = KerasRegressor(build_fn=create_NN_model, shuffle=True, verbose=VERBOSE, **params)
#     history = model.fit(train_X, train_y, verbose=VERBOSE, validation_split = 0.15, callbacks=[early_stopping])
    history = model.fit(X_train, y_train, verbose=VERBOSE, validation_split = None)
    train_results.append(test_mae(model)[0])
    test_results.append(test_mae(model)[1])
    report(train_results, test_results)